In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from app.data_processing.utils import *

In [2]:
import pandas as pd

In [156]:
vg = pd.read_csv('../data/pesos_vegetales.csv', index_col = False)
ing = pd.read_csv('../data/ingredientes.csv')
recipes = pd.read_csv('../data/output.csv',sep='|')
bedca = pd.read_csv('../data/bedca.csv')

In [157]:
ing.sort_values(['Total_Grams'], ascending=False)[:30]

,Recipe_id,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams
15844,36774,cerdo,5.0,kilogramos,175,1000.0,5000.0
25890,26668,agua,5.0,litros,766,1000.0,5000.0
20074,22024,agua,4.0,litros,766,1000.0,4000.0
40282,24593,leche,4.0,litros,27,1000.0,4000.0
16108,60464,agua,4.0,litros,766,1000.0,4000.0
43523,32810,agua,4.0,litros,766,1000.0,4000.0
26078,25460,agua,4.0,litros,766,1000.0,4000.0
16388,12064,leche,4.0,litros,27,1000.0,4000.0
10857,26912,pollo,10.0,muslos,242,390.0,3900.0
6347,59351,agua,15.0,vasos,766,250.0,3750.0


In [127]:
na_count = ing['Grams'].isna().groupby(ing['Recipe_id']).sum()
len(na_count[na_count == 0])


2104

In [158]:
def fill_grams(df):
    '''
    NaN weight values filler function.
        -The ingredients present in the weights dataset are substituted accordingly.
        -For the remaining ingredients: weight(ingredient) = mean(weight(ingredient))
    '''
    result = df.copy()
    result['Index'] = df.index
    original_columns = list(result.columns)

    result['Grams'] = result['Grams'].replace(0,np.nan)
    weights = result[(result['Grams'].isna())]
    
    weights = pd.merge(weights, vg, how='inner', on='Ingrediente')

    weights['Grams'] = weights['Peso x Unidad']
    weights['Unidad'] = 'unidad'
    weights['Total_Grams'] = weights['Grams'].multiply(weights['Cantidad'], axis = 'index')
    
    weights = weights[original_columns]
    weights.set_index(weights['Index'], inplace=True)
    
    result.loc[weights['Index'],:] = weights
    
    #añadir nº comensales
    result = pd.merge(result,recipes['Num_comensales'], how='left', left_on='Recipe_id', right_index=True)
    result['Num_comensales'] = result['Num_comensales']#.fillna(1)
    #result['Total_Grams'] = result['Total_Grams'].divide(result['Num_comensales']).round(5)
    
    result[['Grams','Total_Grams']] = result.groupby(['Ingrediente'])[['Grams','Total_Grams']].transform(lambda x: x.fillna(x.mean()).round(5))
    na_count = result['Grams'].isna().groupby(result['Recipe_id']).sum()
    
    missing = list(na_count[na_count != 0].index)
    result = result[~result['Recipe_id'].isin(missing)]
    
    return result.drop(['Index'], axis=1)

In [159]:
result = fill_grams(ing) #ing[ing['Recipe_id'].isin([46254,70787])])
len(result)


/Users/sheniakuchumova/miniconda3/envs/lpro/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sheniakuchumova/miniconda3/envs/lpro/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sheniakuchumova/miniconda3/envs/lpro/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sheniakuchumova/miniconda3/envs/lpro/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sheniakuchumova/miniconda3/envs/lpro/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sheniaku

51961

In [181]:
result[result['Recipe_id']==27]

,Recipe_id,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams,Num_comensales
35071,27,patata,1.0,sobre,643,16.0,16.0,4.0
35072,27,aceite,1.0,lata,409,330.0,330.0,4.0
35073,27,tomate frito,NaN,None,929,17.0,100.0,4.0
35074,27,huevo,1.0,unidad,150,65.0,65.0,4.0
35075,27,pan rallado,NaN,None,507,17.0,100.0,4.0
35076,27,aceite de oliva,NaN,None,410,2.0,20.0,4.0


In [217]:
new_df = result[['Recipe_id','Total_Grams','Num_comensales']].copy()
new_df = new_df.groupby('Recipe_id', as_index=True).agg({'Total_Grams': 'sum',
                                'Num_comensales': 'mean'})
new_df = new_df.reset_index()
new_df

,Recipe_id,Total_Grams,Num_comensales
0,1,168.0,4.0
1,5,186.0,6.0
2,8,842.5,4.0
3,10,1100.0,1.0
4,27,631.0,4.0
...,...,...,...
7181,74177,1400.0,NaN
7182,74184,248.0,NaN
7183,74194,1609.5,NaN
7184,74195,80.0,NaN


In [218]:
new_df = new_df.merge(recipes[['Id','Categoria']],left_on='Recipe_id', right_on = 'Id')

In [221]:
new_df.drop(['Id'],axis=1,inplace=True)

In [222]:
new_df

,Recipe_id,Total_Grams,Num_comensales,Categoria
0,1,168.0,4.0,Cócteles y bebidas
1,5,186.0,6.0,Recetas de Verduras
2,8,842.5,4.0,Cócteles y bebidas
3,10,1100.0,1.0,Cócteles y bebidas
4,27,631.0,4.0,Recetas de Pescado
...,...,...,...,...
7181,74177,1400.0,NaN,Recetas de Postres
7182,74184,248.0,NaN,Recetas de Postres
7183,74194,1609.5,NaN,Recetas de Aves y caza
7184,74195,80.0,NaN,Recetas de Postres


In [223]:
new_df.to_csv('../data/n_comensales_train.csv')

In [95]:
na_count = result['Total_Grams'].isna().groupby(result['Recipe_id']).sum()
na_count[na_count != 0].index


Int64Index([], dtype='int64', name='Recipe_id')